In [1]:
# -- installs gradio for creating and launching the interface

!pip install --upgrade gradio

# -- installs the pytorch deep learning library 

!pip install torch

# -- installs the ultralytics library to enable accessing the YOLO model

!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 9.6 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.24.7
    Uninstalling huggingface-hub-0.24.7:
      Successfully uninstalled huggingface-hub-0.24.7
   ━━━━

In [2]:
# -- imports the operating system library for file and directory operations

import os

# -- imports the installed gradio library for creating the interface

import gradio as gr

# -- imports the installed pytorch library used by YOLO internally during inferencing

import torch

# -- imports the Image module for handling an manipulating image files

from PIL import Image

# -- imports the gdown library for downloading files directly from the google drives

import gdown

# -- imports the YOLOv8 model for the vehicle detection

from ultralytics import YOLO  

# -- imports the transforms module for image processing
import torchvision.transforms as transforms


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
# Program Code for downloading the trained model from the google drive
file_id = "1lneKF0MFYEJIFMx_L4hMg1UFGeD7EhnR"
url = f"https://drive.google.com/uc?id={file_id}"
output = "best.pt"

# Checks for the presence of the model file and IF NOT, downloads it
if not os.path.exists(output):
    gdown.download(url, output, quiet=False)
    print("Downloaded model file.")
else:
    print("Model file already exists.")

# Loads the YOLOv8 model
try:
    model = YOLO(output)
    print("YOLOv8 model loaded successfully.")
except Exception as e:
    print(f"Error loading YOLOv8 model: {e}")


Downloading...
From: https://drive.google.com/uc?id=1lneKF0MFYEJIFMx_L4hMg1UFGeD7EhnR
To: /content/best.pt
100%|██████████| 22.5M/22.5M [00:00<00:00, 126MB/s]


Downloaded model file.
YOLOv8 model loaded successfully.


In [4]:
# This function detects and counts vehicles from a single image using the trained model.

def detect_vehicles(image):
    try:
        # Performs the YOLOv8 inference on an input image
        results = model.predict(image)

        # Initializes a dictionary to store counts for specific vehicles
        vehicle_counts = {"car": 0, "truck": 0, "motorcycle": 0, "bus": 0}

        # Loops through the detection results and counts relevant vehicles
        for result in results:
            detections = result.boxes
            for box in detections:
                label = model.names[int(box.cls)]
                if label in vehicle_counts:
                    vehicle_counts[label] += 1

        # Computes the total number of vehicles and allocates time dynamically
        total_vehicles = sum(vehicle_counts.values())
        allocated_time = max(10, total_vehicles * 2) # specificies a minimum of 10 seconds

        return vehicle_counts, allocated_time
    except Exception as e:
        print(f"Error during detection: {e}")
        return {"error": str(e)}, 0


In [8]:
# This function processes four lane images to detect vehicles, allocate signal times, and prioritize lanes based on traffic density.

def process_lane_images(image1, image2, image3, image4):
    results = []
    total_time = 0
    lane_vehicle_counts = []  # stores total vehicles and allocated time per lane.

    # Iterates through each of the four lanes
    for i, image in enumerate([image1, image2, image3, image4], start=1):
        vehicle_counts, allocated_time = detect_vehicles(image)
        total_vehicles = sum(vehicle_counts.values())  # Sum of all vehicles in the lane

        # Stores the results for each lane
        results.append({
            "Lane": f"Lane {i}",
            "Vehicle Counts": vehicle_counts,
            "Allocated Time": f"{allocated_time} seconds"
        })

        # Keeps track for prioritization
        lane_vehicle_counts.append((f"Lane {i}", total_vehicles, allocated_time))
        total_time += allocated_time

    # Sorts lanes by the number of detected vehicles
    prioritized_lanes = sorted(lane_vehicle_counts, key=lambda x: x[1])

    # Creates a formatted string showing prioritized lane order
    prioritized_lanes_result = "Prioritized Lanes (from least vehicles to most):\n" + "\n".join(
        [f"{lane}: {count} vehicles, {time} seconds allocated" for lane, count, time in prioritized_lanes]
    )

    # Returns the final structured output.
    return results, f"Total Allocated Time: {total_time} seconds", prioritized_lanes_result


In [9]:
inputs = [gr.Image(type="pil") for _ in range(4)] # the four image inputs for four lanes
outputs = [
    gr.JSON(label="Lane-wise Details"),       # displays the results per lane
    gr.Textbox(label="Total Allocated Time"), # shows the total signal time
    gr.Textbox(label="Prioritized Lanes")     # dislplays lanes by vehicle count
]

# Custom CSS for styling for the interface
custom_css = """
/* Main interface background */
body, .container { background-color: black; color: yellow; }

/* Titles and description */
h1, h2, .container p { color: black; }

/* Input image holders with black and yellow stripes */
.gradio-image-upload .container, .image-upload, .gr-box, .gradio-container {
    border: 3px solid yellow;
    color: yellow;
}

.gradio-container.gradio-container-5-4-0 .contain button {
    color: white;
    background-image: repeating-linear-gradient(
        135deg,
        yellow 0%,
        yellow 10%,
        black 10%,
        black 20%
    );
    background-size: 700px 700px;
}

/* JSON and Textbox outputs */
.output-text, .output-json { color: yellow; background-color: #333; border: 1px solid yellow; }

/* Button styling */
button {
    background-color: yellow;
    color: red;
    border: 2px solid black;
    font-weight: bold;
}
button:hover {
    background-color: black;
    color: yellow;
}
"""

# Launching of the gradio interface
interface = gr.Interface(
    fn=process_lane_images, # executes on submit
    inputs=inputs, # the four lane image inputs
    outputs=outputs, # the JSON and text outputs
    title="Traffic Management System", # app title
    description="Upload 4 images representing 4 lanes to detect vehicles and allocate time to each lane.",
    css=custom_css # applies custom UI styling
)

interface.launch() # launches the gradio interface


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f9b568439bf8e53b91.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
